In [3]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F 

class biLSTMMultitaskPoolModel(nn.Module):
    """マルチタスク用LSTMモデル

    心象ラベルとしてsentiment(7段階)を使用。誤差関数はMSELossを想定。
    """

    def __init__(self, D_i, D_h, D_o, n_classes=3, dropout=0.5):
        super(biLSTMMultitaskPoolModel, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(input_size=D_i, hidden_size=D_h, bidirectional=True, batch_first=True)

        self.linear_persona1 = nn.Linear(D_h*6, D_o)
        self.linear_sentiment1 = nn.Linear(D_h*2, D_o)
        self.linear_persona2 = nn.Linear(D_o, 5)
        self.linear_sentiment2 = nn.Linear(D_o, n_classes)
        self.sigmoid = nn.Sigmoid() 
        self.hidden_size = D_h

    def forward(self, x):
        out, hc = self.lstm(x)
        bsz = out.size()[0]
        seq_len = out.size()[1]
        h_max = F.adaptive_max_pool2d(
            out.view(bsz, 1, seq_len, -1),
            (1, 2 * self.hidden_size)
        ).squeeze(1)
        h_avg = F.adaptive_avg_pool2d(
            out.view(bsz, 1, seq_len, -1),
            (1, 2 * self.hidden_size)
        ).squeeze(1)

        print(out) 
        print(h_max) 
        print(h_avg)


        hc = torch.cat([hc[0][0], hc[0][1]], dim=-1)

        hc = torch.cat([hc, h_max.squeeze(1), h_avg.squeeze(1)], dim=-1)

        h_persona = F.relu(self.linear_persona1(hc))
        h_sentiment = F.relu(self.linear_sentiment1(out))
        h_persona = self.dropout(h_persona)
        h_sentiment = self.dropout(h_sentiment)
        y_persona = self.linear_persona2(h_persona)
        y_sentiment = self.linear_sentiment2(h_sentiment)
        y_persona = self.sigmoid(y_persona)

        return y_persona, y_sentiment

In [17]:
model = biLSTMMultitaskPoolModel(7, 5, 3) 

input = torch.rand((2, 3, 7)) 

_ , _ = model(input) 


tensor([[[ 0.0078,  0.0539,  0.0390,  0.1251,  0.1123,  0.1127, -0.2935,
           0.1447,  0.3564, -0.2187],
         [ 0.0165,  0.0852,  0.0765,  0.1262,  0.0787,  0.2279, -0.2344,
           0.0389,  0.1995, -0.2337],
         [ 0.0264,  0.0170,  0.1415,  0.1332,  0.0625,  0.2402, -0.1851,
           0.0191,  0.0116, -0.1384]],

        [[-0.0487,  0.0196, -0.0344,  0.0167,  0.0423,  0.1375, -0.1548,
          -0.0431,  0.2219, -0.2609],
         [ 0.0189,  0.0847,  0.0596,  0.0619,  0.0328,  0.1100, -0.2172,
           0.0088,  0.2489, -0.1989],
         [-0.0524,  0.0550, -0.0174,  0.0583,  0.1054,  0.0321, -0.0891,
           0.0022,  0.1097, -0.1067]]], grad_fn=<TransposeBackward0>)
tensor([[[ 0.0264,  0.0852,  0.1415,  0.1332,  0.1123,  0.2402, -0.1851,
           0.1447,  0.3564, -0.1384]],

        [[ 0.0189,  0.0847,  0.0596,  0.0619,  0.1054,  0.1375, -0.0891,
           0.0088,  0.2489, -0.1067]]], grad_fn=<SqueezeBackward1>)
tensor([[[ 0.0169,  0.0521,  0.0856,  0.1282, 

In [19]:
(-0.0487 + 0.0189 - 0.0524) / 3

-0.027399999999999997